# Clasificando noticias

In [7]:
#!pip install spacy
#!python -m spacy download es
#!pip install stop_words
#!pip install dplython

## Librerias

In [8]:
import spacy
from stop_words import get_stop_words
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
#from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd 
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction) 

## Cargamos noticias

Los datos vienen de distintas versiones digitales de los periodicos Eltiempo, Portafolio, Clarin, El mundo, El pais, 20minutos y Expansion. Recopilamos alrededor de 900 noticias en temas de deportes, politica, economia y tecnologia. Para cada noticia se ha recolectado la categoria, la url y el contenido en texto. Estos datos se encuentran en una base de datos HIVE.

In [ ]:
#Coloque aqui Credenciales HIVE

In [9]:
# The code was removed by DSX for sharing.

In [11]:
from ingest.Connectors import Connectors
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

HiveloadOptions = { Connectors.Hive.HOST                        : hive_credentials["host"],
                      Connectors.Hive.PORT                      : hive_credentials["port"],
                      Connectors.Hive.DATABASE                  : hive_credentials["database"],
                      Connectors.Hive.USERNAME                  : hive_credentials["username"],
                      Connectors.Hive.PASSWORD                  : hive_credentials["password"],
                      Connectors.Hive.SOURCE_TABLE_NAME         : "noticias_sr"}

hive_df = sqlContext.read.format("com.ibm.spark.discover").options(**HiveloadOptions).load()
hive_df.printSchema()
hive_df.show(10)

root
 |-- categoria: string (nullable = true)
 |-- url: string (nullable = true)
 |-- contenido: string (nullable = true)

+---------+--------------------+--------------------+
|categoria|                 url|           contenido|
+---------+--------------------+--------------------+
| economia|http://www.eltiem...|Un crecimiento de...|
| economia|http://www.eltiem...|Con los 14 despid...|
| economia|http://www.eltiem...|En comparación co...|
| economia|http://www.eltiem...|Residir en el ext...|
| economia|http://www.eltiem...|En marzo, el tope...|
| economia|http://www.eltiem...|                    |
| economia|http://www.eltiem...|Quitarle los tres...|
| economia|http://www.eltiem...|Colombia ascendió...|
| economia|http://www.eltiem...|Durante este miér...|
| economia|http://www.eltiem...|Las contingencias...|
+---------+--------------------+--------------------+
only showing top 10 rows



para procesar los datos preferimos pasarlos al entorno "local". Pasamos de HIVE a Pandas

In [12]:
news_df = hive_df.toPandas()

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/socket.py:647: ResourceWarning: unclosed <socket.socket fd=73, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 48168), raddr=('127.0.0.1', 41057)>
  self._sock = None


Quitamos espacios y saltos de linea a los extremos de url y texto

In [13]:
def safestrip(x):
    if x is None:
        return ''
    else:
        return x.strip()


def vecstrip(series):
    return series.map(lambda x: safestrip(x))

In [14]:
news_df = (news_df >> 
  mutate(url=vecstrip(X.url), contenido = vecstrip(X.contenido)))

Filtramos las filas que no tienen texto

In [15]:
news_df = news_df.query('contenido != ""')

Demos un vistazo a las noticias

In [16]:
news_df.head()

,categoria,url,contenido
0,economia,http://www.eltiempo.com/economia/sectores/cart...,"Un crecimiento de 7,7 por ciento tuvo el saldo..."
1,economia,http://www.eltiempo.com/economia/sectores/desp...,Con los 14 despidos que se produjeron en la te...
2,economia,http://www.eltiempo.com/economia/sectores/dese...,En comparación con la tasa de desempleo de ene...
3,economia,http://www.eltiempo.com/economia/finanzas-pers...,Residir en el exterior no es un obstáculo para...
4,economia,http://www.eltiempo.com/economia/sector-financ...,"En marzo, el tope máximo que tendrán las tasas..."


Podemos ver cuantas noticias tenemos en cada categoria

In [17]:
news_df.groupby('categoria').count()

,url,contenido
categoria,,
deportes,988,988
economia,460,460
politica,217,217
tecnologia,367,367


Es claro que tenemos cierto imbalance en los datos, mas adelante veremos si esto cohibe los resultados de los algoritmos.

# Desarrollo del algoritmo

Vamos a desarrollar un modelo que aprenda a clasificar noticias usando las palabras que se encuentran en los articulos.

En este momento los datos se encuentran limpios, siguen siendo el texto de una noticia habitual que leeriamos. Debemos transformar este texto a una representacon que un computador/algoritmo pueda usar para realizar predicciones.

## Separamos los datos en train y test

El primer paso que debemos realizar con los datos es separarlos en dos conjuntos. Un de entrenamiento que el algoritmo usara para aprender y uno de prueba que luego de aprender servira para ver que tan bueno es el algoritmo.

In [18]:
news_train, news_test, cat_train, cat_test = train_test_split(
    news_df.drop("categoria", axis = 1), news_df["categoria"], test_size=0.25, random_state=42)

## Tokenizer con spaCy

Ahora pasamos a la transformacion del texto plano a una representacion que algoritmos de machine learning puedan manejar.
Escogemos bag of words, una de las mas simples, pero utiles que existen. En orden de obtener la representacion, primero debemos pasar a traves de cada texto y separarlo en tokens.

In [19]:
#import spacy

nlp = spacy.load('es')

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/gpfs/fs01/user/se1f-2003f1257f20bb-7f578b9e48a9/.local/lib/python3.5/site-packages/msgpack_numpy.py:179: PendingDeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpacker.unpack(stream, encoding=encoding, **kwargs)


Aca definimos tokens que no aportan a la diferenciacion de categorias, como signos de puntuacion y palabras muy usadas como "el", "a", etc. Estos seran removidos al extraer los tokens de los articulos.

In [20]:
#from stop_words import get_stop_words

STOPLIST = get_stop_words('es')

#import string
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

WHITES = ["", " ", "\n", "\n\n"]

Esta funcion se encarga de separar el texto en tokens y ademas de hacer un proceso llamado lematizacion.

In [21]:
def tokenizeText(sample):

    # get the tokens using spaCy
    raw_tokens = nlp(sample)

    # lemmatize
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in raw_tokens]

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    
    tokens = [tok for tok in tokens if tok not in WHITES]
    
    #print(tokens[0:2])
    
    # remove large strings of whitespace
    #while "" in tokens:
    #    tokens.remove("")
    #while " " in tokens:
    #    tokens.remove(" ")
    #while "\n" in tokens:
    #    tokens.remove("\n")
    #while "\n\n" in tokens:
    #    tokens.remove("\n\n")

    return tokens

In [22]:
tokenizeText("Hola, mi nombre es David")

['hola', 'nombrar', 'ser', 'david']

Teniendo una forma de separar en tokens, podemos generar la representacion de conteos de bag-of-words. PAra esto hacemos uso de la libreria scikit-learn. Como parametro le pasamos la funcion que definimos para hacer la tokenizacion.

In [23]:
vectorizer = CountVectorizer(tokenizer = tokenizeText, ngram_range=(1,1))

Al aplicar la funcion fit_transform sobre los datos de entrenamiento, el vectorizador aprende el vocabulario y devuelve la representacion en conteos de cada uno de los articulos.

In [24]:
train_counts = vectorizer.fit_transform(news_train.contenido)

La variable train_counts es una gran matriz donde cada columna representa un token y cada fila un articulo. En la celda esta el conteo de cuantas veces dicha palabra aparece en el articulo.

In [25]:
train_counts.shape

(1524, 27370)

## Conteos y Tf-Idf

La representacion de conteos tiene algunos problemas. Por ejemplo, que pasa con articulos mas largos?, tambien, que pasa con palabras que todos los articulos comparten.

Una mejor representacion es la representacion Tf-Idf. En scikit-learn la calculamos de igual manera que hicimos con los conteos.

In [26]:
tf_transformer = TfidfTransformer(use_idf=True)
train_tfidf = tf_transformer.fit_transform(train_counts)

## Modelo de clasificacion en scikit-learn

Compararemos dos algoritmos populares, NaiveBayes y SVM(sgd).

In [27]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)
nb = MultinomialNB()

Realizamos el entrenamiento

In [28]:
sgd_mod = sgd.fit(train_tfidf, cat_train)
nb_mod = nb.fit(train_tfidf, cat_train)

Probamos con los datos de test, primero Naive Bayes y luego SVM

Transformamos los datos de test para que queden como tf-idf, lo que los modelos esperan.

In [29]:
test_counts = vectorizer.transform(news_test.contenido)
test_tfidf = tf_transformer.transform(test_counts)

Entrenamiento de modelos

In [30]:
sgd_test_preds = sgd_mod.predict(test_tfidf)
nb_test_preds = nb_mod.predict(test_tfidf)

Ahora, revisamos resultados para los modelos

* Naive bayes

In [31]:
accuracy_score(cat_test, nb_test_preds)

0.77952755905511806

In [32]:
confusion_matrix(cat_test, nb_test_preds, labels=['economia','deportes','politica', 'tecnologia'])

array([[ 83,  38,   0,   4],
       [  0, 237,   0,   0],
       [  7,  48,   0,   1],
       [  3,  11,   0,  76]])

* SVM (SGD)

In [33]:
accuracy_score(cat_test, sgd_test_preds)

0.96062992125984248

In [34]:
confusion_matrix(cat_test, sgd_test_preds, labels=['economia','deportes','politica', 'tecnologia'])

array([[114,   4,   1,   6],
       [  1, 236,   0,   0],
       [  4,   2,  50,   0],
       [  2,   0,   0,  88]])

### Vamos a probar con noticias de hoy

In [35]:
noticia_1 = """De acuerdo con el fallo del Tribunal Superior de Bogotá el expresidente y hoy senador Álvaro Uribe Velez debe retractarse de las afirmaciones deshonrosas que realizó a través de su cuenta de Twitter contra el periodista Daniel Coronell. Los hechos ocurrieron el 10 de febrero de 2018 cuando el congresista escribió el siguiente mensaje: "Daniel Coronel procede con una actitud mafiosa para hacer daño electoral, sus negocios con narcotráfico siguen impunes y me ha demandado porque pienso que sería extraditable". Para el Tribunal, se deben proteger los derechos al buen nombre y honra del comunicador, los cuales fueron vulnerados por Uribe al publicar este mensaje.

(En contexto: La pelea entre Álvaro Uribe y Daniel Coronell)

Dicho trino se dio tras la publicación de la columna del periodista en la Revista Semana "Aquí no hay muertos" en la que reveló un correo del ex paramilitar Diego Fernando Murillo, mejor conocido como Don Berna, en el que señalaría a Uribe como el responsable de la muerte del empresario colombiano Pedro Juan Moreno.

Uno de los principales argumentos de este Tribunal tiene que ver con el gran número de seguidores con que cuentan, tanto el senador como el periodista, en sus diversas redes sociales y la magnitud del daño moral que causó el mensaje del congresista. El trino habría tenido como único fin "producir daños a la integridad moral en su doble acepción: buen nombre y honra, hechas públicas sin justificación constitucionalmente válida", aseguró el Tribunal. Por este motivo el magistrado Jairo José Agudelo consideró que se debía hacer la retractación.

Recordemos que Álvaro Uribe Velez ya había pasado por una situación similar. En agosto del año pasado el mismo Tribunal ordenó al expresidente rectificar las acusaciones hechas en contra de Daniel Samper Ospina, cuando se refirió a él como un "violador de niños". En su momento, el abogado defensor de Uribe, Jaime Lombana, argumentó que no había fundamento jurídico alguno para reprochar a Uribe Velez y pidió tumbar el fallo del Tribunal. Sin embargo, el senador terminó retractándose de sus afirmaciones en un evento público en la cuidad de Barranquilla."""

In [36]:
noticia_2 = """Sergio Luis Henao es un hombre que es ejemplo de perseverancia, lucha y entrega. De cada caída que ha sufrido, se ha levantado más fuerte. Aunque los médicos llegaron a pronosticarle que no podría volver a montar en bicicleta, tras fracturarse su rodilla derecha en ocho pedazos, este antioqueño salió adelante. Con el apoyo de su familia ha superado cada una de sus crisis y ahora vive el mejor momento de su carrera. (También puede leer: Sergio Luis, te escribe tu hermano)

El ciclista antioqueño es uno de los referentes del Team Sky y tras ganar dos veces el campeonato nacional de ruta en Colombia y obtener el título de la Paris – Niza 2017, este año buscará repetir esa victoria. Desde este domingo participará en la edición 76 de esta competencia. También estarán los colombianos Esteban Chaves (Mitchelton Scott), Jarlinson Pantano (Trek) y Dayer Quintana (Movistar). (Le puede interesar: Sergio Luis Henao, campeón de la París-Niza 2017)

El Espectador recuerda esta entrevista que le dio el corredor de Rionegro hace un año, después de ganar la Paris – Niza.

¿De dónde viene esa berraquera?

Es de familia, mis padres desde pequeño me inculcaron unos valores muy bonitos. El sacrificio, el luchar por las cosas, el creer. Tengo una familia muy unida, amorosa, y eso lo motiva a uno a luchar y a tener berraquera. Crecí con unas condiciones económicas muy difíciles, como muchas familias colombianas. Nuestro padre con el sueldo de celador de una floristería nos pudo sacar adelante a todos, terminamos el bachillerato y luego cada uno eligió qué hacer de su futuro.

¿Cuándo se decidió por el ciclismo?

Tendría unos 12 años cuando mi papá me regaló la primera bicicleta, un casco y un uniforme. Él en algún momento compitió, así que fue fundamental porque me enseñó a pedalear y me motivó.

¿Qué recuerda de esa primera bicicleta?

La utilicé por primera vez en una carrerita en Rionegro. No me fue tan bien porque tuve una caída y no pude disputar el embalaje final. Pero ahí se empieza a aprender de las caídas, nada es fácil. Recuerdo que era marca Torres, color verde y de aluminio. Creo que en ese momento le debió haber costado a mi papá unos $200.000.

¿Y cuánto tiempo le duró?

Comencé a tener buenos resultados, así que fui mejorando poco a poco. Luego tuve una de carbono, luego otra mejor, hasta que tuve la opción de entrar como juvenil al Orgullo Paisa.

¿En ese proceso formativo en algún momento pensó en dedicarse a otra cosa?

Sí. Yo quería comenzar a ayudar rápidamente en mi casa, a llevar un mercado o cosas a mis hermanos y papás, pero el ciclismo no era muy rentable. Ganaba un sueldo pequeño como juvenil en el Orgullo Paisa y los premios de las carreritas que ganaba no eran muy grandes. Así que veía a primos y personas de mi edad que ganaban más que yo y pensaba en tirar la toalla y buscar otra manera de contribuir en casa.

¿Qué hubiera sido de Sergio Luis sin el ciclismo?

En los momentos más duros, en los que quería tirar todo lejos, me daban ganas como de comprar un taxi y ponerme a trabajar. Pero eso lo piensa uno con la cabeza caliente. Nunca desistí.

Ni siquiera con su primer gran golpe en una carrera en Francia antes del Tour de L’Avenir…

Ni ahí. Afortunadamente en mi proceso de recuperación conocí a mi esposa Carolina Caicedo, quien era fisioterapeuta de Coldeportes. Ella fue fundamental para no dejarme rendir y para motivarme a seguir adelante. En medio del dolor y la crisis, también quedan cosas buenas.

¿Si no es por Carolina se retira del ciclismo en 2012, luego de la caída en la Vuelta a Suiza?

Me habían desahuciado para el ciclismo. La rodilla se me fracturó en ocho pedazos. Fue duro, pero ahí estuvo mi esposa y sin ella hubiese sido imposible salir adelante. Anímicamente estuve muy derrotado, hubo peleas y sufrimiento, pero salimos adelante juntos. Si los médicos me decían que no se podía, ella me decía que sí. Fue una berraca, paciente para aguantarme y me levantó.

¿Pero usted llegó a dudar?

Sí. La verdad es que fueron meses en los que no podía ni caminar y me tocaba depender de alguien para moverme. Además, los expertos no decían nada bueno, así que me desanimaba y pensaba que sería el final de mi carrera.

¿Ahora qué siente cuando se ve esa cicatriz en la rodilla derecha?

La veo y me siento un berraco. Fue algo duro pero que no me venció y ahora cuando camino y veo lo que hago, le doy gracias a Dios y a la vida por cómo me funciona de bien esa rodilla. Esa cicatriz me llena de orgullo.

¿El título del Campeonato Nacional de Ruta en 2017 fue lo que necesitaba para olvidar el golpe en Río 2016?

Eso de Río nunca lo olvidaré. En el momento de la caída sentí muchísimo dolor, pero luego, en el hospital, me dio mucha rabia. Tenía una oportunidad única, pero Dios sabe cómo hace sus cosas. Me impresionó el apoyo de la gente en Bogotá, como que todos habían sentido mi dolor en Río y en la caída de ese día en el campeonato nacional, así que querían que fuera yo el ganador. Di todo de mí y afortunadamente pude sentir por primera vez lo que era un triunfo de este tipo.

Y luego llegó la París-Niza 2017…

Ser el campeón nacional de ruta me llenó de cosas positivas. Me motivó muchísimo y creo que llevar el tricolor en la París-Niza hizo que sintiera algo diferente. Uno desde que comienza en el ciclismo aspira a ganar carreras de este tipo y la verdad, cuando lo logré, ni me lo creía.

Justo estos triunfos llegaron tras el nacimiento de Emanuel, su primogénito…

Emanuel es mi motivación, es querer ser su orgullo, su ejemplo. Ser un buen guía para él. Cuando salgo a entrenar o voy a competir y lo dejo solo, le pido a Dios que pueda regresar con salud para verle sus ojitos.

Su gloria ha sido muy sufrida, ¿no?

(Risas) La verdad es que el sufrimiento como que es algo característico en mí. Nada es fácil y cuando se gana así, se disfruta más.

Usted le enseña al pueblo colombiano que de todas las caídas hay que levantarse y seguir luchando, ¿es consciente de eso?

El pueblo colombiano ha sufrido la violencia, la corrupción de nuestros políticos, pero siempre nos levantamos y somos gente berraca, que a pesar de todo es feliz y sale adelante de los momentos más complicados. Es bonito que yo pueda ser ejemplo de eso en el mundo entero. Soy un colombiano que nunca se rinde.

¿Cómo es Chris Froome como jefe?

Él es una excelente persona, un gran líder. Muy sereno, humilde, agradecido con sus compañeros y tiene una gran relación con todos. Él me respeta mucho, yo lo admiro y siempre soy leal con él. Le he aprendido muchas cosas, cómo correr y cómo esforzarse.

¿Le sorprendió que fuera a verlo coronarse campeón de la París-Niza 2017?

Él es así. Sentí mucho orgullo por recibir ese mensaje que me puso en Twitter. Es una persona excepcional y un deportista ejemplar.

¿Cómo asumieron en el Sky su título del campeonato nacional?

Cuando llegué a Europa y vi la camiseta que me diseñaron, me motivó muchísimo porque le dan el valor que es ser el campeón nacional en Colombia. Ellos saben lo duro que es competir y ganar acá y han valorado mucho mi triunfo. Para mí es más que un orgullo llevar a Colombia en el pecho por las carreteras del mundo."""

In [37]:
noticias = [noticia_1, noticia_2]

In [38]:
noticias_counts = vectorizer.transform(noticias)
noticias_tfidf = tf_transformer.transform(noticias_counts)

In [42]:
noticias_preds = sgd_mod.predict(noticias_tfidf)
print(noticias_preds)

['politica' 'deportes']
